In [1]:
import tensorflow as tf
import os
import numpy as np
from model import *
from utility import *

model_name = "DRAGAN_SRResNet_for_IRASUTOYA"

In [2]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.g_bn0 = BatchNormalization(name = 'g_bn0')
        self.g_bn1 = BatchNormalization(name = 'g_bn1')
        
        self.num_res_blocks = 16
        self.num_pixel_CNN_blocks = 3
        
        self.res_bns = []
        for i in range(int(self.num_res_blocks)):
            self.res_bns.append(BatchNormalization(name = "res_%d" % (2*i)))
            self.res_bns.append(BatchNormalization(name = "res_%d" % (2*i+1)))
        
        self.ps_bns = []
        for i in range(int(self.num_pixel_CNN_blocks)):
            self.ps_bns.append(BatchNormalization(name = "ps_%d" % i))

    def __call__(self, z):
        with tf.variable_scope('g', reuse=self.reuse):
            
            # reshape from inputs
            with tf.variable_scope('fc0'):
                #z0 = tf.reshape(z, [-1, self.z_dim])
                fc0 = full_connection_layer(z, 64*16*16, name="fc0")
                fc0 = self.g_bn0(fc0)
                fc0 = tf.nn.relu(fc0)
                fc0 = tf.reshape(fc0, [-1,16,16,64])

            assert fc0.get_shape().as_list()[1:] == [16,16,64]
            
            layers = []
            layers.append(fc0)
            
            for i in range(int(self.num_res_blocks)):
                with tf.variable_scope('res_%d' % (i+1)):
                    res = conv2d_layer(layers[-1], 64, kernel_size=3, strides=1, name="g_conv_res_%d" % (2*i))
                    res = self.res_bns[2*i](res)
                    res = tf.nn.relu(res)

                    res = conv2d_layer(res, 64, kernel_size=3, strides=1, name="g_conv_res_%d" % (2*i+1))
                    res = self.res_bns[2*i+1](res)
                    res = layers[-1] + res
                    layers.append(res)                    

            assert layers[-1].get_shape().as_list()[1:] == [16,16,64]
            
            with tf.variable_scope('conv17'):
                conv17 = conv2d_layer(layers[-1], 64, kernel_size=3, strides=1, name="g_conv_17")
                conv17 = self.g_bn1(conv17)
                conv17 = tf.nn.relu(conv17)
                conv17 = layers[0] + conv17
                layers.append(conv17)

            assert layers[-1].get_shape().as_list()[1:] == [16, 16, 64]

            for i in range(int(self.num_pixel_CNN_blocks)):
                with tf.variable_scope('pixel_CNN_%d' % (i+1)):
                    ps = conv2d_layer(layers[-1], 256, kernel_size=3, strides=1, name="g_conv_ps_%d" % (i))
                    ps = pixel_shuffle_layer(ps, 2, 64)
                    ps = self.ps_bns[i](ps)
                    ps = tf.nn.relu(ps)
                    layers.append(ps)

            assert layers[-1].get_shape().as_list()[1:] == [128, 128, 64]
            
            with tf.variable_scope('output'):
                output = conv2d_layer(layers[-1], 3, kernel_size=9, strides=1, name="output")
                output = tf.nn.sigmoid(output)

            assert output.get_shape().as_list()[1:] == [128, 128, 3]            
            output = tf.reshape(output, [-1, 128*128*3])
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return output

In [3]:
class Discriminator:
    def __init__(self):
        self.reuse = False
        self.d_bn0 = BatchNormalization(name="d_bn0")
        self.d_bn1 = BatchNormalization(name="d_bn1")
        self.d_bn2 = BatchNormalization(name="d_bn2")
        self.d_bn3 = BatchNormalization(name="d_bn3")
        self.d_bn4 = BatchNormalization(name="d_bn4")

    def __call__(self, x):
        with tf.variable_scope('d', reuse=self.reuse):
           
            # convolution x 4
            x = tf.reshape(x, [-1, 128, 128, 3])
            with tf.variable_scope('conv1'):
                conv1 = conv2d_layer(x, 32, kernel_size=4, strides=2, name="conv1")
                conv1 = lrelu(conv1, leak=0.2)

            with tf.variable_scope('res1'):                
                res1 = conv2d_layer(conv1, 32, kernel_size=3, strides=1, name="res1_conv0")
                res1 = lrelu(res1, leak=0.2)
                res1 = conv2d_layer(res1, 32, kernel_size=3, strides=1, name="res1_conv1")
                res1 = res1 + conv1
                res1 = lrelu(res1, leak=0.2)
                
            with tf.variable_scope('res2'):
                res2 = conv2d_layer(res1, 32, kernel_size=3, strides=1, name="res2_conv0")
                res2 = lrelu(res2, leak=0.2)
                res2 = conv2d_layer(res2, 32, kernel_size=3, strides=1, name="res2_conv1")
                res2 = res2 + res1
                res2 = lrelu(res2, leak=0.2)

            with tf.variable_scope('conv2'):
                conv2 = conv2d_layer(res2, 64, kernel_size=4, strides=2, name="conv2")
                conv2 = lrelu(conv2, leak=0.2)

            with tf.variable_scope('res3'):
                res3 = conv2d_layer(conv2, 64, kernel_size=3, strides=1, name="res3_conv0")
                res3 = lrelu(res3, leak=0.2)
                res3 = conv2d_layer(res3, 64, kernel_size=3, strides=1, name="res3_conv1")
                res3 = res3 + conv2
                res3 = lrelu(res3, leak=0.2)            

            with tf.variable_scope('res4'):
                res4 = conv2d_layer(res3, 64, kernel_size=3, strides=1, name="res4_conv0")
                res4 = lrelu(res4, leak=0.2)
                res4 = conv2d_layer(res4, 64, kernel_size=3, strides=1, name="res4_conv1")
                res4 = res4 + res3
                res4 = lrelu(res4, leak=0.2)                    
                
            with tf.variable_scope('conv3'):
                conv3 = conv2d_layer(res4, 128, kernel_size=4, strides=2, name="conv3")
                conv3 = lrelu(conv3, leak=0.2)

            num_res_itr = 3
            layers = []
            layers.append(conv3)
            
            depth = [128, 256, 512, 1024]
            for i in range(int(num_res_itr)):
                with tf.variable_scope('res_%d_1' % (i+1+4)):
                    res = conv2d_layer(layers[-1], depth[i], kernel_size=3, strides=1, name="res_%d" % (5*i+0))
                    res = lrelu(res)
                    res = conv2d_layer(res, depth[i], kernel_size=3, strides=1, name="res_%d" % (5*i+1))
                    res = lrelu(res)
                    res = layers[-1] + res
                    res = lrelu(res)
                layers.append(res)

                with tf.variable_scope('res_%d_2' % (i+1+4)):
                    res = conv2d_layer(layers[-1], depth[i], kernel_size=3, strides=1, name="res_%d" % (5*i+2))
                    res = lrelu(res)
                    res = conv2d_layer(res, depth[i], kernel_size=3, strides=1, name="res_%d" % (5*i+3))
                    res = lrelu(res)
                    res = layers[-1] + res
                    res = lrelu(res)                  
                    
                conv = conv2d_layer(res, depth[i+1], kernel_size=4, strides=2, name="res_%d" % (5*i+4))
                conv = lrelu(conv) 
                layers.append(conv)
                
            disc = full_connection_layer(layers[-1], 1, name="disc")

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')

        return disc

In [ ]:
class GAN:
    def __init__(self):
        self.batch_size = 64
        self.img_size = 128
        self.rand_size = 100
        self.z_size = self.rand_size
        
        self.epochs = 50000
        self.epoch_saveMetrics = 200
        self.epoch_saveSampleImg = 200
        self.epoch_saveParamter = 5000
        self.losses = {"d_loss":[], "g_loss":[]}

        # unrolled counts
        self.steps = 10

        self.X_tr = tf.placeholder(tf.float32, shape=[None, self.img_size*self.img_size*3])
        self.z = tf.placeholder(tf.float32, [None, self.z_size])
        self.X_per = tf.placeholder(tf.float32, shape=[None, self.img_size*self.img_size*3])

        self.g = Generator()
        self.d = Discriminator()
        self.Xg = self.g(self.z)
        #self.pokemon = Pokemon()
        self.irasutoya = IRASUTOYA()
        self.learning_rate = tf.placeholder("float", [])
        self.count = 0
    def loss(self):
        self.count += 1
        tf.summary.image("X_tr", tf.reshape(self.X_tr, [-1, 128, 128, 3]))
        tf.summary.image("Xg", tf.reshape(self.Xg, [-1, 128, 128, 3]))
        disc_tr = self.d(self.X_tr)
        disc_gen = self.d(self.Xg)
        self.disc_tr = disc_tr
        self.disc_gen = disc_gen
        
        #tf.summary.histogram('disc_tr', disc_tr)
        #tf.summary.histogram("disc_gen", disc_gen)

        lambda_gp = 10
       
        loss_g = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_gen, labels=tf.ones_like(disc_gen)))

        diff = self.X_per - self.X_tr
        alpha = tf.random_uniform(shape=[self.batch_size,1,1,1], minval=0., maxval=1.)
        interpolates = self.X_tr + (alpha*diff)
        disc_interplates = self.d(interpolates)
        gradients = tf.gradients(disc_interplates, [interpolates])[0]
        slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1]))
        gradient_penalty = tf.reduce_mean((slopes-1.)**2)

        loss_d_tr = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_tr, labels=tf.ones_like(disc_tr)))
        loss_d_gen = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_gen, labels=tf.zeros_like(disc_gen)))
        loss_d = (loss_d_tr + loss_d_gen)
        loss_d += lambda_gp*gradient_penalty


        tf.summary.scalar("loss_g", loss_g)
        tf.summary.scalar("loss_d", loss_d)
        return loss_g, loss_d

    def train(self):
        # Optimizer
        d_lr = 1e-4
        d_beta1 = 0.5
        g_lr = 1e-4
        g_beta1 = 0.5

        self.L_g, self.L_d = self.loss()

        d_opt = tf.train.AdamOptimizer(learning_rate=d_lr)
        d_train_op = d_opt.minimize(self.L_d, var_list=self.d.variables)
        g_opt = tf.train.AdamOptimizer(learning_rate=g_lr)
        g_train_op = g_opt.minimize(self.L_g, var_list=self.g.variables)

        saver = tf.train.Saver()
        
        config = tf.ConfigProto(
            gpu_options=tf.GPUOptions(
                visible_device_list= "0"
            )
        )
                
        summary = tf.summary.merge_all()

        with tf.Session(config=config) as sess:
            writer = tf.summary.FileWriter('cnn', sess.graph)
            sess.run(tf.global_variables_initializer())

            # preparing noise vec for test
            bs = 100
            test_z = np.random.uniform(-1, 1, size=[bs, self.rand_size])

            lr = 1e-4
            for epoch in range(self.epochs):
                if epoch > 2000:
                    lr = 5e-5
                if epoch > 4000:
                    lr = 1e-5

                # visualize generated images during training
                if epoch % self.epoch_saveSampleImg == 0:
                    img = sess.run(self.Xg, feed_dict={self.z: test_z})
                    img = np.reshape(img, [-1, 128, 128, 3])
                    img += 0.5
                    save_imgs(model_name, img, name=str(epoch))

                for step in range(self.steps):
                    # extract images for training
                    #rand_index = np.random.randint(0, self.dataset.shape[0], size=self.batch_size)
                    #X_mb, Y_mb = self.dataset[rand_index, :].astype(np.float32)
                    #X_mb = self.dtd.extract(self.batch_size, self.img_size)                   
                    X_mb, Y_mb = self.irasutoya.extract(self.batch_size, self.img_size)

                    #X_mb = self.pokemon.extract(self.batch_size, self.img_size)
                    X_mb = np.reshape(X_mb, [-1, self.img_size*self.img_size*3])
                    X_mb_per = X_mb + 0.5*np.std(X_mb)*np.random.random(X_mb.shape)
                    #save_imgs(model_name, X_mb, name=str(epoch)+"_d")

                    rand = np.random.uniform(-1, 1, size=[self.batch_size, self.rand_size])

                    # train Discriminator
                    _, d_loss_value = sess.run([d_train_op, self.L_d], feed_dict={
                        self.X_tr: X_mb,
                        self.z:rand,
                        self.X_per: X_mb_per,
                    })

                # train Generator
                disc_tr, disc_gen, _, g_loss_value, w_summary = sess.run([self.disc_tr, self.disc_gen, g_train_op, self.L_g, summary], feed_dict={
                    self.X_tr: X_mb,
                    self.z:rand,
                    self.X_per: X_mb_per,
                })

                writer.add_summary(w_summary, epoch)

                # append loss value for visualizing
                self.losses["d_loss"].append(np.sum(d_loss_value))
                self.losses["g_loss"].append(np.sum(g_loss_value))
                
                # print epoch
                if epoch % 1 == 0:
                    print('epoch:{0}, d_loss:{1}, g_loss: {2}, disc_tr: {3}, disc_gen: {4} '.format(epoch, d_loss_value, g_loss_value, disc_tr, disc_gen))
                
                # visualize loss
                if epoch % self.epoch_saveMetrics == 0:
                    save_metrics(model_name, self.losses, epoch)


                # save model parameters 
                if epoch % self.epoch_saveParamter == 0:
                    dir_path = "model_" + model_name
                    if not os.path.isdir(dir_path):
                        os.makedirs(dir_path)

                    saver.save(sess, dir_path + "/" + str(epoch) + ".ckpt")

In [ ]:
gan = GAN()
gan.train()

init IRASUTOYA
1839


/home/ujitoko/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:80: DeprecationWarning: PyUnicode_AsEncodedObject() is deprecated; use PyUnicode_AsEncodedString() to encode from str to bytes or PyCodec_Encode() for generic encoding


epoch:0, d_loss:10.552452087402344, g_loss: 12.07174301147461, disc_tr: [[ 0.57227427]
 [ 0.57704484]], disc_gen: [[-12.01258755]
 [-12.13088799]] 
epoch:1, d_loss:9.780306816101074, g_loss: 260.17816162109375, disc_tr: [[ 36.99403  ]
 [ 14.4256382]], disc_gen: [[-260.03598022]
 [-260.32034302]] 
epoch:2, d_loss:23.921293258666992, g_loss: 8.051680924836546e-05, disc_tr: [[ 12.40630341]
 [ 12.96924591]], disc_gen: [[ 9.46440601]
 [ 9.39095116]] 
epoch:3, d_loss:11.37394905090332, g_loss: 0.7275923490524292, disc_tr: [[-0.0733645 ]
 [-0.06950997]], disc_gen: [[-0.06763718]
 [-0.06784932]] 
epoch:4, d_loss:11.410528182983398, g_loss: 0.8009722232818604, disc_tr: [[-0.25259614]
 [-0.20173454]], disc_gen: [[-0.20616636]
 [-0.20412742]] 
epoch:5, d_loss:11.387401580810547, g_loss: 0.7488482594490051, disc_tr: [[-0.10363361]
 [-0.09999483]], disc_gen: [[-0.10798794]
 [-0.10893709]] 
epoch:6, d_loss:11.363054275512695, g_loss: 0.7207698822021484, disc_tr: [[-0.00057218]
 [-0.01119529]], disc_